In [1]:
using LowLevelFEM, LinearAlgebra, SparseArrays

[ Info: Precompiling LowLevelFEM [6171b9fb-adbf-4751-adb9-5faded75de07] (cache misses: include_dependency fsize change (2), wrong dep version loaded (2), incompatible header (14))

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
┌ Info: Skipping precompilation due to precompilable error. Importing LowLevelFEM [6171b9fb-adbf-4751-adb9-5faded75de07].
└   exception = Error when precompiling module, potentially caused by a __precompile__(false) declaration in the module.

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodE

In [2]:
gmsh.initialize()
gmsh.open("cube1.geo")

mat = Material("cube", E=2e2)
prob = Problem([mat], type=:VectorField)

Info    : Reading 'cube1.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.000673167s, CPU 0.000675s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 20%] Meshing surface 2 (Transfinite)
Info    : [ 40%] Meshing surface 3 (Transfinite)
Info    : [ 60%] Meshing surface 4 (Transfinite)
Info    : [ 70%] Meshing surface 5 (Transfinite)
Info    : [ 90%] Meshing surface 6 (Transfinite)
Info    : Done meshing 2D (Wall 0.000313111s, CPU 0.000312s)
Info    : Meshing 3D

Problem("cube1", :VectorField, 3, 3, Material[Material("cube", :Hooke, 200.0, 0.3, 115.38461538461537, 76.92307692307692, 166.66666666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 6859, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [3]:
F = TensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1]);

In [4]:
#Fleft = nodesToElements(elementsToNodes(F), onPhysicalGroup="left")

#Kext = externalTangentFollowerTL(prob; F=Fleft, traction_phName="left", t_spatial=[1, 0, 0])

In [5]:
suppX = BoundaryCondition("left", ux=0, uy=0, uz=0)
suppY = BoundaryCondition("bottom", uy=0)
suppZ = BoundaryCondition("rear", uz=0)


load = BoundaryCondition("right", fy=(x, y, z) -> -(z - 0.5) * 30, fz=(x, y, z) -> (y - 0.5) * 30)

BoundaryCondition("right", nothing, nothing, nothing, nothing, nothing, nothing, nothing, var"#3#4"(), var"#5#6"(), nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [6]:
μ = mat.μ
λ = mat.λ

115.38461538461537

In [7]:
f_ext0 = loadVector(prob, [load])

nodal VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [8]:
showDoFResults(f_ext0, name="f")

1

In [9]:
u = vectorField(prob, "cube", [0, 0, 0])

nodal VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [10]:
F = tensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1])

nodal TensorField
[1.0; 0.0; … ; 0.0; 1.0;;]

In [11]:
I = tensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1])

nodal TensorField
[1.0; 0.0; … ; 0.0; 1.0;;]

In [12]:
P = tensorField(prob, "cube", [0 0 0; 0 0 0; 0 0 0])
S = tensorField(prob, "cube", [0 0 0; 0 0 0; 0 0 0])

nodal TensorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [13]:
λ0 = λ
μ0 = μ
C1 = [λ0+2μ0 λ0 λ0 0 0 0;
    λ0 λ0+2μ0 λ0 0 0 0;
    λ0 λ0 λ0+2μ0 0 0 0;
    0 0 0 μ0 0 0;
    0 0 0 0 μ0 0;
    0 0 0 0 0 μ0]

6×6 Matrix{Float64}:
 269.231  115.385  115.385   0.0      0.0      0.0
 115.385  269.231  115.385   0.0      0.0      0.0
 115.385  115.385  269.231   0.0      0.0      0.0
   0.0      0.0      0.0    76.9231   0.0      0.0
   0.0      0.0      0.0     0.0     76.9231   0.0
   0.0      0.0      0.0     0.0      0.0     76.9231

## Innen indul az iteráció

In [16]:
for i in 1:10

    Kmat = materialTangentMatrix(prob, F=F, C=C1)

    Kgeo = initialStressMatrix(prob, stress=S)

    Kint = Kmat + Kgeo
    Fright = nodesToElements(elementsToNodes(F), onPhysicalGroup="right")

    Kext = externalTangentFollower(prob, [load], F=Fright)
    f_int = internalForceVector(prob, P)
    K = Kint - Kext
    FF = nodesToElements(elementsToNodes(F), onPhysicalGroup="right")
    f_ext = loadVector(prob, [load], F=FF)
    Δu = solveField(
        K,
        f_ext - f_int;
        support=[suppX]
    )

    fixed = constrainedDoFs(prob, [suppX])
    free = freeDoFs(prob, [suppX])
    #norm(Δu.a[:, 1])
    r = f_ext - f_int
    println("‖r_free‖  = ", norm(r.a[free, 1]))
    println("‖r_fixed‖ = ", norm(r.a[fixed, 1]))
    println("‖r‖ = ", norm(r.a[:, 1]))
    println("‖Δu‖ = ", norm(Δu.a[:, 1]))

    u += Δu
    showDoFResults(u)
    F = I + u ∘ ∇
    E = 0.5 * (F' * F - I)
    S = λ * trace(E) * I + 2μ * E
    P = F * S
    #P = λ * trace(F - I) * I + 2μ * (F - I)

    if norm(Δu.a[:, 1]) < 0.01
        break
    end

end

‖r_free‖  = 0.7372768252552513
‖r_fixed‖ = 0.0
‖r‖ = 0.7372768252552513
‖Δu‖ = 54.215657539454675
‖r_free‖  = 1698.4951921405277
‖r_fixed‖ = 0.5919368122012353
‖r‖ = 1698.4952952874723
‖Δu‖ = 34.26678099261648
‖r_free‖  = 468.41311315750585
‖r_fixed‖ = 0.8263261383601883
‖r‖ = 468.41384201664374
‖Δu‖ = 10.546767490147475
‖r_free‖  = 117.14118378431888
‖r_fixed‖ = 0.5408920073521569
‖r‖ = 117.14243254498
‖Δu‖ = 21.367172344603443
‖r_free‖  = 25.336996083429263
‖r_fixed‖ = 0.44544056322882164
‖r‖ = 25.34091134563
‖Δu‖ = 6.892718243931763
‖r_free‖  = 3.985002413669754
‖r_fixed‖ = 0.38499315062192846
‖r‖ = 4.003556414362056
‖Δu‖ = 2.124663286017278
‖r_free‖  = 1.3371593679620601
‖r_fixed‖ = 0.3554831111660596
‖r‖ = 1.3836052246406838
‖Δu‖ = 3.2713566891324977
‖r_free‖  = 3.6780530691999216
‖r_fixed‖ = 0.3472698284243643
‖r‖ = 3.6944107396965014
‖Δu‖ = 1.8510552307197476
‖r_free‖  = 0.8636740660868873
‖r_fixed‖ = 0.35550446075882425
‖r‖ = 0.9339787546033793
‖Δu‖ = 4.611512549290819
‖r_free‖

In [17]:
openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0
Fontconfig warning: using without calling FcInit()


In [ ]:
gmsh.finalize()